In [1]:
import numpy as np
import matplotlib.pyplot as plt
import moviepy.editor as mpy
import skimage.transform
from IPython.display import Image, display

import tensorflow as tf
import tensorflow_probability as tfp
import tensorflow.keras.losses as kls

pygame 2.2.0 (SDL 2.0.22, Python 3.10.16)
Hello from the pygame community. https://www.pygame.org/contribute.html


ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5701:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2664:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No su

In [2]:
gpus = tf.config.list_physical_devices("GPU")
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.set_visible_devices(gpus[0], "GPU")

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)

        logical_gpus = tf.config.list_logical_devices("GPU")
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

1 Physical GPUs, 1 Logical GPUs


2024-12-17 23:23:22.179339: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-17 23:23:22.215982: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-17 23:23:22.216046: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-17 23:23:22.220112: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:887] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2024-12-17 23:23:22.220170: I external/local_xla/xla/stream_executor

In [3]:
import os

os.environ["SDL_VIDEODRIVER"] = (
    "dummy"  # this line make pop-out window not appear
)
from ple.games.flappybird import FlappyBird
from ple import PLE

game = FlappyBird()
env = PLE(game, fps=30, display_screen=False)  # environment interface to game
env.reset_game()

test_game = FlappyBird()
test_env = PLE(test_game, fps=30, display_screen=False)
test_env.reset_game()

couldn't import doomish
Couldn't import doom


libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
libpng warning: iCCP: known incorrect sRGB profile
ALSA lib confmisc.c:855:(parse_card) cannot find card '0'
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_card_inum returned error: No such file or directory
ALSA lib confmisc.c:422:(snd_func_concat) error evaluating strings
ALSA lib conf.c:5178:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1334:(snd_func_refer) error evaluating name
ALSA lib conf.c:5178:(_sn

In [4]:
path = "./movie_f"
if not os.path.exists(path):
    os.makedirs(path)

In [5]:
hparas = {
    "image_size": 84,
    "num_stack": 4,
    "action_dim": len(env.getActionSet()),
    "hidden_size": 256,
    "lr": 0.0001,
    "gamma": 0.99,
    "lambda": 0.95,
    "clip_val": 0.2,
    "ppo_epochs": 8,
    "test_epochs": 1,
    "num_steps": 512,
    "mini_batch_size": 64,
    "target_reward": 200,
    "max_episode": 30000,
}

In [6]:
# Please do not modify this method
def make_anim(images, fps=60, true_image=False):
    duration = len(images) / fps

    def make_frame(t):
        try:
            x = images[int(len(images) / duration * t)]
        except:
            x = images[-1]

        if true_image:
            return x.astype(np.uint8)
        else:
            return ((x + 1) / 2 * 255).astype(np.uint8)

    clip = mpy.VideoClip(make_frame, duration=duration)
    clip.fps = fps

    return clip

In [7]:
def preprocess_screen(screen):
    screen = skimage.transform.rotate(screen, -90, resize=True)
    screen = screen[:400, :]
    screen = skimage.transform.resize(
        screen, [hparas["image_size"], hparas["image_size"], 1]
    )
    return screen.astype(np.float32)


def frames_to_state(input_frames):
    if len(input_frames) == 1:
        state = np.concatenate(input_frames * 4, axis=-1)
    elif len(input_frames) == 2:
        state = np.concatenate(
            input_frames[0:1] * 2 + input_frames[1:] * 2, axis=-1
        )
    elif len(input_frames) == 3:
        state = np.concatenate(input_frames + input_frames[2:], axis=-1)
    else:
        state = np.concatenate(input_frames[-4:], axis=-1)

    return state

In [8]:
class ActorCriticNetwork(tf.keras.Model):
    def __init__(self, hparas):
        super().__init__()

        self.feature_extractor = tf.keras.Sequential(
            [
                # Convolutional Layers
                tf.keras.layers.Conv2D(filters=32, kernel_size=8, strides=4),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(filters=64, kernel_size=4, strides=2),
                tf.keras.layers.ReLU(),
                tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1),
                tf.keras.layers.ReLU(),
                # Embedding Layers
                tf.keras.layers.Flatten(),
                tf.keras.layers.Dense(hparas["hidden_size"]),
                tf.keras.layers.ReLU(),
            ]
        )

        # Actor Network
        self.actor = tf.keras.layers.Dense(
            hparas["action_dim"], activation="softmax"
        )
        # Critic Network
        self.critic = tf.keras.layers.Dense(1, activation=None)

    def call(self, input):
        x = self.feature_extractor(input)
        action_logits = self.actor(x)
        value = self.critic(x)
        return action_logits, value

In [9]:
class Agent:
    def __init__(self, hparas):
        self.gamma = hparas["gamma"]
        self.optimizer = tf.keras.optimizers.Adam(learning_rate=hparas["lr"])
        self.actor_critic = ActorCriticNetwork(hparas)
        self.clip_pram = hparas["clip_val"]

    def ppo_iter(
        self, mini_batch_size, states, actions, log_probs, returns, advantage
    ):
        batch_size = states.shape[0]
        for _ in range(batch_size // mini_batch_size):
            rand_ids = tf.convert_to_tensor(
                np.random.randint(0, batch_size, mini_batch_size),
                dtype=tf.int32,
            )
            yield tf.gather(states, rand_ids), tf.gather(
                actions, rand_ids
            ), tf.gather(log_probs, rand_ids), tf.gather(
                returns, rand_ids
            ), tf.gather(
                advantage, rand_ids
            )

    def ppo_update(
        self,
        ppo_epochs,
        mini_batch_size,
        states,
        actions,
        log_probs,
        discount_rewards,
        advantages,
    ):
        total_actor_loss = 0
        total_critic_loss = 0
        for _ in range(ppo_epochs):
            for (
                state,
                action,
                old_log_probs,
                reward,
                advantage,
            ) in self.ppo_iter(
                mini_batch_size,
                states,
                actions,
                log_probs,
                discount_rewards,
                advantages,
            ):
                reward = tf.expand_dims(reward, axis=-1)

                with tf.GradientTape() as tape:
                    prob, value = self.actor_critic(state, training=True)
                    dist = tfp.distributions.Categorical(
                        probs=prob, dtype=tf.float32
                    )
                    entropy = tf.math.reduce_mean(dist.entropy())
                    new_log_probs = dist.log_prob(action)

                    # PPO ratio
                    ratio = tf.math.exp(new_log_probs - old_log_probs)
                    surr1 = ratio * advantage
                    surr2 = (
                        tf.clip_by_value(
                            ratio, 1.0 - self.clip_pram, 1.0 + self.clip_pram
                        )
                        * advantage
                    )

                    actor_loss = (
                        tf.math.negative(
                            tf.math.reduce_mean(tf.math.minimum(surr1, surr2))
                        )
                        - 0.1 * entropy
                    )
                    critic_loss = 0.5 * tf.math.reduce_mean(
                        kls.mean_squared_error(reward, value)
                    )

                    total_loss = actor_loss + critic_loss

                # single optimizer
                grads = tape.gradient(
                    total_loss, self.actor_critic.trainable_variables
                )
                self.optimizer.apply_gradients(
                    zip(grads, self.actor_critic.trainable_variables)
                )

                total_actor_loss += actor_loss
                total_critic_loss += critic_loss
        return total_actor_loss, total_critic_loss

In [10]:
# https://arxiv.org/pdf/1506.02438.pdf
# Equation 16
def compute_gae(rewards, masks, values, gamma, LAMBDA):
    gae = 0
    returns = []
    for i in reversed(range(len(rewards))):
        delta = rewards[i] + gamma * values[i + 1] * masks[i] - values[i]
        gae = delta + gamma * LAMBDA * masks[i] * gae
        returns.append(gae + values[i])

    returns.reverse()
    return returns

# Testing Environment

In [11]:
def test_reward(test_env, agent):
    total_reward = 0
    # Reset the environment
    test_env.reset_game()
    input_frames = [preprocess_screen(test_env.getScreenGrayscale())]

    while not test_env.game_over():

        state = frames_to_state(input_frames)
        state = tf.expand_dims(state, axis=0)
        prob, value = agent.actor_critic(state)

        action = np.argmax(prob[0].numpy())
        reward = test_env.act(test_env.getActionSet()[action])
        total_reward += reward

        input_frames.append(preprocess_screen(test_env.getScreenGrayscale()))

    return total_reward

# Training

In [12]:
agent = Agent(hparas)
max_episode = hparas["max_episode"]
test_per_n_episode = 10
force_save_per_n_episode = 1000
early_stop_reward = 10

start_s = 0
best_reward = -5.0

checkpoint = tf.train.Checkpoint(
    actor_critic=agent.actor_critic,
    optimizer=agent.optimizer,
)

# Load from old checkpoint
# checkpoint.restore('ckpt_dir/ckpt-?')

In [13]:
ep_reward = []
total_avgr = []
early_stop = False
avg_rewards_list = []

env.reset_game()

for s in range(0, max_episode):
    if early_stop == True:
        break

    rewards = []
    states = []
    actions = []
    log_probs = []
    masks = []
    values = []

    display_frames = [env.getScreenRGB()]
    input_frames = [preprocess_screen(env.getScreenGrayscale())]

    for step in range(hparas["num_steps"]):

        state = frames_to_state(input_frames)
        state = tf.expand_dims(state, axis=0)
        prob, value = agent.actor_critic(state)

        dist = tfp.distributions.Categorical(probs=prob[0], dtype=tf.float32)
        action = dist.sample(1)
        log_prob = dist.log_prob(action)

        reward = env.act(env.getActionSet()[int(action.numpy())])

        done = env.game_over()

        states.append(state)
        actions.append(action)
        values.append(value[0])
        log_probs.append(log_prob)
        rewards.append(tf.convert_to_tensor(reward, dtype=tf.float32))
        masks.append(tf.convert_to_tensor(1 - int(done), dtype=tf.float32))

        display_frames.append(env.getScreenRGB())
        input_frames.append(preprocess_screen(env.getScreenGrayscale()))

        if done:
            env.reset_game()
            input_frames = [preprocess_screen(env.getScreenGrayscale())]

    _, next_value = agent.actor_critic(state)
    values.append(next_value[0])

    returns = compute_gae(
        rewards, masks, values, hparas["gamma"], hparas["lambda"]
    )

    returns = tf.concat(returns, axis=0)
    log_probs = tf.concat(log_probs, axis=0)
    values = tf.concat(values, axis=0)
    states = tf.concat(states, axis=0)
    actions = tf.concat(actions, axis=0)
    advantage = returns - values[:-1]

    a_loss, c_loss = agent.ppo_update(
        hparas["ppo_epochs"],
        hparas["mini_batch_size"],
        states,
        actions,
        log_probs,
        returns,
        advantage,
    )
    print(
        "[Episode %d]  Actor loss: %.5f, Critic loss: %.5f"
        % (s, a_loss, c_loss)
    )

    if s % test_per_n_episode == 0:
        # test agent hparas['test_epochs'] times to get the average reward
        avg_reward = np.mean(
            [
                test_reward(test_env, agent)
                for _ in range(hparas["test_epochs"])
            ]
        )
        print(
            "Test average reward is %.1f, Current best average reward is %.1f\n"
            % (avg_reward, best_reward)
        )
        avg_rewards_list.append(avg_reward)

        if avg_reward > best_reward:
            best_reward = avg_reward
            agent.actor_critic.save(
                "./save/Actor/model_actor_{}_{}".format(s, avg_reward),
                save_format="tf",
            )
            checkpoint.save(file_prefix="./save/checkpoints/ckpt")

    if s % force_save_per_n_episode == 0:
        agent.actor_critic.save(
            "./save/Actor/model_actor_{}_{}".format(s, avg_reward),
            save_format="tf",
        )
        checkpoint.save(file_prefix="./save/checkpoints/ckpt")
        clip = make_anim(display_frames, fps=60, true_image=True).rotate(-90)
        clip.write_videofile(
            "movie_f/{}_demo-{}.webm".format("Lab15", s), fps=60
        )
        display(
            clip.ipython_display(fps=60, autoplay=1, loop=1, maxduration=120)
        )

    if best_reward >= early_stop_reward:
        early_stop = True

2024-12-17 23:23:31.329111: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:454] Loaded cuDNN version 8904
2024-12-17 23:23:42.110671: I external/local_xla/xla/service/service.cc:168] XLA service 0x3eebbcf0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-12-17 23:23:42.110711: I external/local_xla/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 3060 Ti, Compute Capability 8.6
2024-12-17 23:23:42.114604: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1734449022.175164   14166 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


[Episode 0]  Actor loss: 87.90213, Critic loss: 65.48260
Test average reward is -5.0, Current best average reward is -5.0

INFO:tensorflow:Assets written to: ./save/Actor/model_actor_0_-5.0/assets


INFO:tensorflow:Assets written to: ./save/Actor/model_actor_0_-5.0/assets


[MoviePy] >>>> Building video movie_f/Lab15_demo-0.webm
[MoviePy] Writing video movie_f/Lab15_demo-0.webm


100%|█████████▉| 513/514 [00:04<00:00, 103.32it/s]

...

INFO:tensorflow:Assets written to: ./save/Actor/model_actor_1130_-1.0/assets
  reward = env.act(env.getActionSet()[int(action.numpy())])



[Episode 1131]  Actor loss: -0.71589, Critic loss: 2.68339
[Episode 1132]  Actor loss: 7.66646, Critic loss: 2.57870
[Episode 1133]  Actor loss: 0.54086, Critic loss: 1.83839
[Episode 1134]  Actor loss: 4.28962, Critic loss: 1.26187
[Episode 1135]  Actor loss: -8.65089, Critic loss: 0.86870
[Episode 1136]  Actor loss: -12.59831, Critic loss: 2.14328
[Episode 1137]  Actor loss: -0.76224, Critic loss: 0.69113
[Episode 1138]  Actor loss: -4.37694, Critic loss: 0.45660
[Episode 1139]  Actor loss: -6.74597, Critic loss: 0.88501
[Episode 1140]  Actor loss: -1.05721, Critic loss: 0.66264
Test average reward is -3.0, Current best average reward is -1.0

[Episode 1141]  Actor loss: -12.15967, Critic loss: 1.21418
[Episode 1142]  Actor loss: -7.83993, Critic loss: 0.88949
[Episode 1143]  Actor loss: -7.24127, Critic loss: 1.26311
[Episode 1144]  Actor loss: -26.65961, Critic loss: 4.18686
[Episode 1145]  Actor loss: -4.11811, Critic loss: 2.39153
[Episode 1146]  Actor loss: -4.19401, Critic loss

INFO:tensorflow:Assets written to: ./save/Actor/model_actor_1480_3.0/assets
  reward = env.act(env.getActionSet()[int(action.numpy())])



[Episode 1481]  Actor loss: -10.16659, Critic loss: 3.35980
[Episode 1482]  Actor loss: 2.31942, Critic loss: 3.40860
[Episode 1483]  Actor loss: 2.83230, Critic loss: 3.31518
[Episode 1484]  Actor loss: 3.02837, Critic loss: 2.65266
[Episode 1485]  Actor loss: -20.01434, Critic loss: 4.11631
[Episode 1486]  Actor loss: -12.50602, Critic loss: 2.83692
[Episode 1487]  Actor loss: -7.31221, Critic loss: 4.54891
[Episode 1488]  Actor loss: -20.50581, Critic loss: 2.30457
[Episode 1489]  Actor loss: 1.34535, Critic loss: 5.37482
[Episode 1490]  Actor loss: -6.27008, Critic loss: 2.41275
Test average reward is -5.0, Current best average reward is 3.0

[Episode 1491]  Actor loss: -11.16527, Critic loss: 5.69942
[Episode 1492]  Actor loss: -6.03604, Critic loss: 3.90337
[Episode 1493]  Actor loss: -1.67294, Critic loss: 3.10027
[Episode 1494]  Actor loss: -13.49863, Critic loss: 2.55067
[Episode 1495]  Actor loss: 7.27832, Critic loss: 7.28027
[Episode 1496]  Actor loss: 5.28908, Critic loss:

INFO:tensorflow:Assets written to: ./save/Actor/model_actor_1930_13.0/assets


In [14]:
clip = make_anim(display_frames, fps=60, true_image=True).rotate(-90)
clip.write_videofile(
    "movie_f/{}_demo-{}.webm".format("Lab15", s), fps=60
)
display(
    clip.ipython_display(fps=60, autoplay=1, loop=1, maxduration=120)
)

[MoviePy] >>>> Building video movie_f/Lab15_demo-1931.webm
[MoviePy] Writing video movie_f/Lab15_demo-1931.webm


100%|█████████▉| 513/514 [00:05<00:00, 94.22it/s] 

[MoviePy] Done.
[MoviePy] >>>> Video ready: movie_f/Lab15_demo-1931.webm 




100%|█████████▉| 513/514 [00:00<00:00, 766.31it/s]


# Report

The training process is different. We first simulates the game and record the state, action and reward. Then we train the model with the recorded data.

It took nearly a million steps to train the model to get a average reward of 13.

Compared to the last lab, DQN achives higher rewards in same training time. However, with DQN we can see that the rewards are not stable. 
Before getting the average reward of 13, in most of the episodes the rewards are still -5, and the best average reward is 3.

We can see, in my final video, the agent somtimes gets to high rewards, while somtimes it just bumped into the first pipe.
